In [1]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs

# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36"
}

In [2]:
leagueTables = [
    "https://www.transfermarkt.com/premier-league/tabelle/wettbewerb/GB1/saison_id/2024",
    "https://www.transfermarkt.com/laliga/tabelle/wettbewerb/ES1/saison_id/2024",
    "https://www.transfermarkt.com/ligue-1/tabelle/wettbewerb/FR1/saison_id/2024",
    "https://www.transfermarkt.com/bundesliga/tabelle/wettbewerb/L1/saison_id/2024",
    "https://www.transfermarkt.com/serie-a/tabelle/wettbewerb/IT1/saison_id/2024",
]

listClub = pd.DataFrame()
for link in leagueTables:
    re = req.get(link, headers=headers)
    so = bs(re.content, "html.parser")
    tb = so.select_one('table:contains("Goals")')
    innerListClub = pd.DataFrame(
        {
            "Ranking": [(i.text) for i in tb.select("tbody td:nth-of-type(1)")],
            "Mini Team Crest": [
                (i["src"]) for i in tb.select("tbody td:nth-of-type(2) a img")
            ],
            "Team": [(i["title"]) for i in tb.select("tbody td:nth-of-type(2) a img")],
            "Team Link": [
                ("https://www.transfermarkt.com" + i["href"] + "/plus/1")
                for i in tb.select("tbody td:nth-of-type(3) a:nth-of-type(1)")
            ],
            "MP": [(i.text) for i in tb.select("tbody td:nth-of-type(4)")],
            "W": [(i.text) for i in tb.select("tbody td:nth-of-type(5)")],
            "D": [(i.text) for i in tb.select("tbody td:nth-of-type(6)")],
            "L": [(i.text) for i in tb.select("tbody td:nth-of-type(7)")],
            "Goals": [(i.text) for i in tb.select("tbody td:nth-of-type(8)")],
            "+/-": [(i.text) for i in tb.select("tbody td:nth-of-type(9)")],
            "Pts": [(i.text) for i in tb.select("tbody td:nth-of-type(10)")],
        }
    )

    def decideLeague(x):
        if link == leagueTables[0]:
            x = "Premier League"
            return x
        elif link == leagueTables[1]:
            x = "LaLiga"
            return x
        elif link == leagueTables[2]:
            x = "Ligue 1"
            return x
        elif link == leagueTables[3]:
            x = "Bundesliga"
            return x
        elif link == leagueTables[4]:
            x = "Serie A"
            return x

    innerListClub["League"] = innerListClub.apply(decideLeague, axis=1)

    listClub = pd.concat([listClub, innerListClub], axis=0)


def clubCrestBig(x):
    return x["Mini Team Crest"].replace("tiny", "head")


listClub["Big Team Crest"] = listClub.apply(clubCrestBig, axis=1)
listClub.insert(1, "Big Team Crest", listClub.pop("Big Team Crest"))


def teamLink(x):
    return x["Team Link"].replace("/spielplan/", "/kader/")


listClub["Team Link"] = listClub.apply(teamLink, axis=1)


def teamLinkSecondFirst(x):
    return x["Team Link"].replace("/kader/", "/leistungsdaten/")


listClub["Team Link 2"] = listClub.apply(teamLinkSecondFirst, axis=1)


def teamLinkSecondSecond(x):
    if x["League"] == "Premier League":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/GB1%262024/")
    elif x["League"] == "LaLiga":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/ES1%262024/")
    elif x["League"] == "Ligue 1":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/FR1%262024/")
    elif x["League"] == "Bundesliga":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/L1%262024/")
    elif x["League"] == "Serie A":
        return x["Team Link 2"].replace("/saison_id/2024/", "/reldata/IT1%262024/")


listClub["Team Link 2"] = listClub.apply(teamLinkSecondSecond, axis=1)


listClub["Pts/MP"] = listClub["Pts"].astype(int) / listClub["MP"].astype(int)
listClub["Pts/MP"] = listClub["Pts/MP"].round(2)
listClubDetails = listClub.sort_values(by="Team", ascending=True)
finalListClub = listClubDetails.reset_index(drop=True)

finalListClub

/opt/hostedtoolcache/Python/3.13.3/x64/lib/python3.13/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


,Ranking,Big Team Crest,Mini Team Crest,Team,Team Link,MP,W,D,L,Goals,+/-,Pts,League,Team Link 2,Pts/MP
0,16,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,1.FC Heidenheim 1846,https://www.transfermarkt.com/1-fc-heidenheim-...,30,6,4,20,32:60,-28,22,Bundesliga,https://www.transfermarkt.com/1-fc-heidenheim-...,0.73
1,13,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,1.FC Union Berlin,https://www.transfermarkt.com/1-fc-union-berli...,30,9,8,13,30:44,-14,35,Bundesliga,https://www.transfermarkt.com/1-fc-union-berli...,1.17
2,6,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,1.FSV Mainz 05,https://www.transfermarkt.com/1-fsv-mainz-05/k...,30,13,8,9,48:36,12,47,Bundesliga,https://www.transfermarkt.com/1-fsv-mainz-05/l...,1.57
3,9,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,AC Milan,https://www.transfermarkt.com/ac-mailand/kader...,33,14,9,10,51:38,13,51,Serie A,https://www.transfermarkt.com/ac-mailand/leist...,1.55
4,20,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,AC Monza,https://www.transfermarkt.com/ac-monza/kader/v...,33,2,9,22,25:57,-32,15,Serie A,https://www.transfermarkt.com/ac-monza/leistun...,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,17,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,VfL Bochum,https://www.transfermarkt.com/vfl-bochum/kader...,30,5,5,20,29:62,-33,20,Bundesliga,https://www.transfermarkt.com/vfl-bochum/leist...,0.67
92,12,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,VfL Wolfsburg,https://www.transfermarkt.com/vfl-wolfsburg/ka...,30,10,9,11,53:47,6,39,Bundesliga,https://www.transfermarkt.com/vfl-wolfsburg/le...,1.30
93,5,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,Villarreal CF,https://www.transfermarkt.com/fc-villarreal/ka...,31,14,10,7,55:42,13,52,LaLiga,https://www.transfermarkt.com/fc-villarreal/le...,1.68
94,17,https://tmssl.akamaized.net//images/wappen/hea...,https://tmssl.akamaized.net//images/wappen/tin...,West Ham United,https://www.transfermarkt.com/west-ham-united/...,33,9,9,15,37:55,-18,36,Premier League,https://www.transfermarkt.com/west-ham-united/...,1.09


In [3]:
listPlayer = pd.DataFrame()
for l in finalListClub["Team Link"]:
    r = req.get(
        l,
        headers=headers,
    )
    s = bs(r.content, "html.parser")
    t = s.find("table", class_="items")
    rows = s.select("tbody tr")
    
    innerListPlayer = pd.DataFrame(
        {
            "Player": [(i["title"]) for i in t.select("tbody td:nth-of-type(1) img")],
            "Position": [
                (i.text)
                for i in t.select("tbody td:nth-of-type(2) tr td:nth-of-type(1)")
            ][1::2],
            "Age": [(i.text) for i in t.select("tbody td:nth-of-type(3)")],
            "Height": [(i.text) for i in t.select("tbody td:nth-of-type(5)")],
            "Foot": [(i.text) for i in t.select("tbody td:nth-of-type(6)")],
            "Market Value": [(i.text) for i in t.select("tbody .rechts.hauptlink")],
        }
    )

    dataNation = [[td["title"].strip() for td in row.select("td:nth-of-type(4) img")] for row in rows]
    filteredDataNation = [sublist for sublist in dataNation if sublist]
    dataNationFlag = [[td["src"].strip() for td in row.select("td:nth-of-type(4) img")] for row in rows]
    filteredDataNationFlag = [sublist for sublist in dataNationFlag if sublist]

    filteredDataNationList = []
    for i in filteredDataNation:
        filteredDataNationList.append(i[0])

    filteredDataNationFlagList = []
    for i in filteredDataNationFlag:
        filteredDataNationFlagList.append(i[0])

    innerListPlayer["Nation"] = filteredDataNationList
    innerListPlayer["Nation Flag Link"] = filteredDataNationFlagList

    teams = s.find_all(
        class_="data-header__headline-wrapper data-header__headline-wrapper--oswald"
    )
    teamName = [team.get_text(strip=True) for team in teams]

    def addTeamName(c):
        c = teamName[0]
        return c

    innerListPlayer["Team"] = innerListPlayer.apply(addTeamName, axis=1)

    listPlayer = pd.concat([listPlayer, innerListPlayer], axis=0)

listPlayer["Age"] = listPlayer["Age"].str[-3:-1]
listPlayer["Position"] = listPlayer["Position"].replace("\n", "", regex=True)
listPlayer["Position"] = listPlayer["Position"].str.strip()
listPlayer.index = range(0, len(listPlayer))

listPlayer

,Player,Position,Age,Height,Foot,Market Value,Nation,Nation Flag Link,Team
0,Kevin Müller,Goalkeeper,34,"1,90m",right,€1.00m,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846
1,Frank Feller,Goalkeeper,21,"1,83m",right,€500k,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846
2,Vitus Eicher,Goalkeeper,34,"1,91m",left,€250k,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846
3,Paul Tschernuth,Goalkeeper,23,"1,86m",left,€100k,Austria,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846
4,Benedikt Gimber,Centre-Back,28,"1,87m",left,€3.00m,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846
...,...,...,...,...,...,...,...,...,...
2575,Matheus Cunha,Centre-Forward,25,"1,83m",right,€55.00m,Brazil,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers
2576,Jørgen Strand Larsen,Centre-Forward,25,"1,93m",right,€27.00m,Norway,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers
2577,Hee-chan Hwang,Centre-Forward,29,"1,77m",right,€17.00m,"Korea, South",https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers
2578,Sasa Kalajdzic,Centre-Forward,27,"2,00m",right,€5.00m,Austria,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers


In [4]:
marketValueOne = listPlayer[listPlayer["Market Value"].str.contains("k")]
listPlayer.drop(
    listPlayer[listPlayer["Market Value"].str.contains("k")].index,
    inplace=True,
)

marketValueSecond = marketValueOne[
    marketValueOne["Market Value"].str.contains("\xa0\xa0")
]
marketValueOne.drop(
    marketValueOne[marketValueOne["Market Value"].str.contains("\xa0\xa0")].index,
    inplace=True,
)

/tmp/ipykernel_2016/3745614528.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketValueOne.drop(


In [5]:
marketValueSeventh = pd.DataFrame()
marketValueSeventh["Market Value"] = marketValueOne["Market Value"].str[1:-1]
marketValueSeventh["Market Value"] = pd.to_numeric(marketValueSeventh["Market Value"])

marketValueSeventh

,Market Value
1,500
2,250
3,100
7,300
13,200
...,...
2523,900
2524,700
2531,700
2552,200


In [6]:
valueList = []
for value in marketValueSeventh["Market Value"]:
    if value < 100 and value > 0:
        value = 0.1
        valueList.append(value)
    else:
        value = round((value / 1000), 1)
        valueList.append(value)

valueList

[0.5,
 0.2,
 0.1,
 0.3,
 0.2,
 0.2,
 0.2,
 0.5,
 0.7,
 0.3,
 0.6,
 0.1,
 0.3,
 0.2,
 0.8,
 0.1,
 0.8,
 0.5,
 0.8,
 0.1,
 0.8,
 0.8,
 0.6,
 0.2,
 0.5,
 0.3,
 0.8,
 0.3,
 0.1,
 0.6,
 0.1,
 0.8,
 0.1,
 0.1,
 0.2,
 0.2,
 0.5,
 0.5,
 0.3,
 0.1,
 0.5,
 0.5,
 0.9,
 0.7,
 0.8,
 0.7,
 0.8,
 0.8,
 0.8,
 0.2,
 0.5,
 0.3,
 0.8,
 0.3,
 0.7,
 0.2,
 0.3,
 0.8,
 0.2,
 0.6,
 0.5,
 0.1,
 0.5,
 0.2,
 0.8,
 0.5,
 0.5,
 0.2,
 0.4,
 0.9,
 0.8,
 0.2,
 0.6,
 0.3,
 0.3,
 0.5,
 0.5,
 0.9,
 0.6,
 0.6,
 0.6,
 0.8,
 0.6,
 0.6,
 0.1,
 0.7,
 0.7,
 0.9,
 0.8,
 0.9,
 0.9,
 0.8,
 0.7,
 0.6,
 0.1,
 0.1,
 0.2,
 0.4,
 0.3,
 0.5,
 0.6,
 0.2,
 0.1,
 0.4,
 0.9,
 0.8,
 0.2,
 0.3,
 0.8,
 0.3,
 0.5,
 0.5,
 0.1,
 0.5,
 0.2,
 0.5,
 0.2,
 0.7,
 0.8,
 0.1,
 0.5,
 0.1,
 0.5,
 0.3,
 0.2,
 0.1,
 0.6,
 0.1,
 0.9,
 0.6,
 0.2,
 0.5,
 0.8,
 0.1,
 0.5,
 0.5,
 0.3,
 0.3,
 0.6,
 0.1,
 0.8,
 0.8,
 0.5,
 0.7,
 0.1,
 0.8,
 0.3,
 0.5,
 0.8,
 0.4,
 0.2,
 0.2,
 0.1,
 0.9,
 0.3,
 0.2,
 0.1,
 0.9,
 0.4,
 0.8,
 0.5,
 0.3,
 0.4,
 0.2,
 0.8,
 0.8,
 0.7

In [7]:
marketValueSeventh["Market Value"] = valueList
marketValueSeventh

,Market Value
1,0.5
2,0.2
3,0.1
7,0.3
13,0.2
...,...
2523,0.9
2524,0.7
2531,0.7
2552,0.2


In [8]:
marketValueEighth = pd.DataFrame()
marketValueEighth["Market Value"] = marketValueSecond["Market Value"].str[1:-3]
marketValueEighth["Market Value"] = pd.to_numeric(marketValueEighth["Market Value"])

marketValueEighth

,Market Value
2475,700


In [9]:
otherValueList = []
for v in marketValueEighth["Market Value"]:
    if v < 100 and v > 0:
        v = 0.1
        otherValueList.append(v)
    else:
        v = round((v / 1000), 1)
        otherValueList.append(v)

otherValueList

[0.7]

In [10]:
marketValueEighth["Market Value"] = otherValueList
marketValueEighth

,Market Value
2475,0.7


In [11]:
marketValueOne.drop(columns=["Market Value"], inplace=True)
marketValueSecond.drop(columns=["Market Value"], inplace=True)

/tmp/ipykernel_2016/2085958102.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketValueOne.drop(columns=["Market Value"], inplace=True)
/tmp/ipykernel_2016/2085958102.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketValueSecond.drop(columns=["Market Value"], inplace=True)


In [12]:
marketValueThird = pd.concat([marketValueOne, marketValueSeventh], axis=1)
marketValueNinth = pd.concat([marketValueSecond, marketValueEighth], axis=1)

marketValueTenth = pd.concat([marketValueThird, marketValueNinth], axis=0)

In [13]:
marketValueTenth

,Player,Position,Age,Height,Foot,Nation,Nation Flag Link,Team,Market Value
1,Frank Feller,Goalkeeper,21,"1,83m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.5
2,Vitus Eicher,Goalkeeper,34,"1,91m",left,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.2
3,Paul Tschernuth,Goalkeeper,23,"1,86m",left,Austria,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.1
7,Thomas Keller,Centre-Back,25,"1,86m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.3
13,Luka Janes,Defensive Midfield,21,"1,83m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.2
...,...,...,...,...,...,...,...,...,...
2524,Lukasz Fabianski,Goalkeeper,40,"1,90m",right,Poland,https://tmssl.akamaized.net//images/flagge/ver...,West Ham United,0.7
2531,Aaron Cresswell,Left-Back,35,"1,70m",left,England,https://tmssl.akamaized.net//images/flagge/ver...,West Ham United,0.7
2552,Tom King,Goalkeeper,30,"1,94m",right,Wales,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers,0.2
2579,Nathan Fraser,Centre-Forward,20,"1,88m",right,Ireland,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers,0.6


In [14]:
marketValueFourth = listPlayer[listPlayer["Market Value"].str.contains("m")]
listPlayer.drop(
    listPlayer[listPlayer["Market Value"].str.contains("m")].index,
    inplace=True,
)

marketValueFifth = marketValueFourth[marketValueFourth["Market Value"].str.contains("\xa0\xa0")]
marketValueFourth.drop(
    marketValueFourth[marketValueFourth["Market Value"].str.contains("\xa0\xa0")].index,
    inplace=True,
)

/tmp/ipykernel_2016/2434364806.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketValueFourth.drop(


In [15]:
listPlayer["Market Value"] = 0.0
listPlayer.insert(4, "Market Value", listPlayer.pop("Market Value"))

In [16]:
listPlayer

,Player,Position,Age,Height,Market Value,Foot,Nation,Nation Flag Link,Team
35,Oluwaseun Ogbemudia,Centre-Back,18,"1,88m",0.0,right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Union Berlin
168,Max Kinsey,Centre-Back,20,"1,84m",0.0,right,England,https://tmssl.akamaized.net//images/flagge/ver...,AFC Bournemouth
189,Tom Negrel,Goalkeeper,22,"1,92m",0.0,right,France,https://tmssl.akamaized.net//images/flagge/ver...,AJ Auxerre
205,Ben Viadere,Attacking Midfield,19,"1,76m",0.0,right,France,https://tmssl.akamaized.net//images/flagge/ver...,AJ Auxerre
295,Oumar Pona,Goalkeeper,18,"1,89m",0.0,right,France,https://tmssl.akamaized.net//images/flagge/ver...,Angers SCO
445,Luca Novodomsky,Goalkeeper,19,-,0.0,,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bayer 04 Leverkusen
563,Niklas Swider,Defensive Midfield,18,"1,88m",0.0,right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Borussia Mönchengladbach
1039,Ylies Aradj,Left-Back,19,"1,79m",0.0,left,France,https://tmssl.akamaized.net//images/flagge/ver...,FC Toulouse
1048,Mathis Saka,Attacking Midfield,18,"1,72m",0.0,right,France,https://tmssl.akamaized.net//images/flagge/ver...,FC Toulouse
1348,Ruben Londja,Right Winger,18,"1,79m",0.0,left,Switzerland,https://tmssl.akamaized.net//images/flagge/ver...,Le Havre AC


In [17]:
marketValueSixth = pd.DataFrame()
marketValueSixth["Market Value"] = marketValueFourth["Market Value"].str[1:-1]
marketValueSixth["Market Value"] = pd.to_numeric(marketValueSixth["Market Value"])

marketValueSixth

,Market Value
0,1.0
4,3.0
5,3.0
6,1.5
8,2.0
...,...
2574,7.0
2575,55.0
2576,27.0
2577,17.0


In [18]:
marketValueEleventh = pd.DataFrame()
marketValueEleventh["Market Value"] = marketValueFifth["Market Value"].str[1:-3]
marketValueEleventh["Market Value"] = pd.to_numeric(marketValueEleventh["Market Value"])

marketValueEleventh

,Market Value


In [19]:
marketValueFourth.drop(columns=["Market Value"], inplace=True)
marketValueFifth.drop(columns=["Market Value"], inplace=True)

/tmp/ipykernel_2016/3646839043.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketValueFourth.drop(columns=["Market Value"], inplace=True)
/tmp/ipykernel_2016/3646839043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  marketValueFifth.drop(columns=["Market Value"], inplace=True)


In [20]:
marketValueTwelfth = pd.concat([marketValueFourth, marketValueSixth], axis=1)
marketValueThirteenth = pd.concat([marketValueFifth, marketValueEleventh], axis=1)

marketValueFourteenth = pd.concat([marketValueTwelfth, marketValueThirteenth], axis=0)

In [21]:
marketValueFourteenth

,Player,Position,Age,Height,Foot,Nation,Nation Flag Link,Team,Market Value
0,Kevin Müller,Goalkeeper,34,"1,90m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,1.0
4,Benedikt Gimber,Centre-Back,28,"1,87m",left,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,3.0
5,Patrick Mainka,Centre-Back,30,"1,94m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,3.0
6,Tim Siersleben,Centre-Back,25,"1,87m",left,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,1.5
8,Jonas Föhrenbach,Left-Back,29,"1,84m",left,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,2.0
...,...,...,...,...,...,...,...,...,...
2574,Pablo Sarabia,Right Winger,32,"1,74m",left,Spain,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers,7.0
2575,Matheus Cunha,Centre-Forward,25,"1,83m",right,Brazil,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers,55.0
2576,Jørgen Strand Larsen,Centre-Forward,25,"1,93m",right,Norway,https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers,27.0
2577,Hee-chan Hwang,Centre-Forward,29,"1,77m",right,"Korea, South",https://tmssl.akamaized.net//images/flagge/ver...,Wolverhampton Wanderers,17.0


In [22]:
finalListPlayer = pd.concat(
    [marketValueTenth, marketValueFourteenth, listPlayer], axis=0
)
finalListPlayer = finalListPlayer.reset_index(drop=True)
finalListPlayer

,Player,Position,Age,Height,Foot,Nation,Nation Flag Link,Team,Market Value
0,Frank Feller,Goalkeeper,21,"1,83m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.5
1,Vitus Eicher,Goalkeeper,34,"1,91m",left,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.2
2,Paul Tschernuth,Goalkeeper,23,"1,86m",left,Austria,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.1
3,Thomas Keller,Centre-Back,25,"1,86m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.3
4,Luka Janes,Defensive Midfield,21,"1,83m",right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,1.FC Heidenheim 1846,0.2
...,...,...,...,...,...,...,...,...,...
2575,Patrice Covic,Attacking Midfield,17,-,right,Croatia,https://tmssl.akamaized.net//images/flagge/ver...,SV Werder Bremen,0.0
2576,Rento Takaoka,Centre-Forward,18,"1,65m",,Japan,https://tmssl.akamaized.net//images/flagge/ver...,Southampton FC,0.0
2577,Noah Jauny,Goalkeeper,20,"1,94m",right,Ireland,https://tmssl.akamaized.net//images/flagge/ver...,Stade Brestois 29,0.0
2578,Elijah Scott,Centre-Back,19,-,right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,US Lecce,0.0


In [23]:
listPlayerDetail = pd.DataFrame()
for li in finalListClub["Team Link 2"]:

    res = req.get(
        li,
        headers=headers,
    )
    sou = bs(res.content, "html.parser")
    tbl = sou.select_one('table:contains("PPG")')
    innerListPlayerDetail = pd.DataFrame(
        {
            "Player": [(i["title"]) for i in tbl.select("tbody td:nth-of-type(1) img")],
            "Position": [
                (i.text)
                for i in tbl.select("tbody td:nth-of-type(2) tr td:nth-of-type(1)")
            ][1::2],
            "Age": [(i.text) for i in tbl.select("tbody td:nth-of-type(3)")],
            "MP": [(i.text) for i in tbl.select("tbody td:nth-of-type(6)")],
            "Gls": [(i.text) for i in tbl.select("tbody td:nth-of-type(7)")],
            "Asts": [(i.text) for i in tbl.select("tbody td:nth-of-type(8)")],
            "YC": [(i.text) for i in tbl.select("tbody td:nth-of-type(9)")],
            "Second YC": [(i.text) for i in tbl.select("tbody td:nth-of-type(10)")],
            "RC": [(i.text) for i in tbl.select("tbody td:nth-of-type(11)")],
            "Subs On": [(i.text) for i in tbl.select("tbody td:nth-of-type(12)")],
            "Min": [(i.text) for i in tbl.select("tbody td:nth-of-type(15)")],
        }
    )

    teams = sou.find_all(
        class_="data-header__headline-wrapper data-header__headline-wrapper--oswald"
    )
    teamNameDetail = [team.get_text(strip=True) for team in teams]
    def addTeamNameDetail(c):
        c = teamNameDetail[0]
        return c
    innerListPlayerDetail["Team"] = innerListPlayerDetail.apply(
        addTeamNameDetail, axis=1
    )

    leagues = sou.find_all(
        class_="data-header__club"
    )
    leagueNameDetail = [league.get_text(strip=True) for league in leagues]
    def addLeagueNameDetail(d):
        d = leagueNameDetail[0]
        return d
    innerListPlayerDetail["League"] = innerListPlayerDetail.apply(
        addLeagueNameDetail, axis=1
    )

    listPlayerDetail = pd.concat([listPlayerDetail, innerListPlayerDetail], axis=0)

/opt/hostedtoolcache/Python/3.13.3/x64/lib/python3.13/site-packages/soupsieve/css_parser.py:876: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


In [24]:
listPlayerDetail = listPlayerDetail.reset_index(drop=True)
listPlayerDetail

,Player,Position,Age,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team,League
0,Kevin Müller,Goalkeeper,34,30,-,-,-,-,-,-,2.700',1.FC Heidenheim 1846,Bundesliga
1,Frank Feller,Goalkeeper,21,Not used during this season,-,-,-,-,-,-,-,1.FC Heidenheim 1846,Bundesliga
2,Vitus Eicher,Goalkeeper,34,Not used during this season,-,-,-,-,-,-,-,1.FC Heidenheim 1846,Bundesliga
3,Paul Tschernuth,Goalkeeper,23,Not in squad during this season,-,-,-,-,-,-,-,1.FC Heidenheim 1846,Bundesliga
4,Omar Traoré,Right-Back,27,26,-,1,5,-,-,-,2.269',1.FC Heidenheim 1846,Bundesliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,Enso González,Left Winger,20,Not in squad during this season,-,-,-,-,-,-,-,Wolverhampton Wanderers,Premier League
3524,Chiquinho,Left Winger,25,1,-,-,-,-,-,1,6',Wolverhampton Wanderers,Premier League
3525,Nathan Fraser,Centre-Forward,20,Not in squad during this season,-,-,-,-,-,-,-,Wolverhampton Wanderers,Premier League
3526,Tom Edozie,Right Winger,18,Not used during this season,-,-,-,-,-,-,-,Wolverhampton Wanderers,Premier League


In [25]:
listPlayerDetail["MP"] = listPlayerDetail["MP"].replace("Not used during this season", 0)
listPlayerDetail["MP"] = listPlayerDetail["MP"].replace("Not in squad during this season", 0)
listPlayerDetail["MP"] = listPlayerDetail["MP"].replace("", 0)
listPlayerDetail["MP"] = listPlayerDetail["MP"].astype(int)

listPlayerDetail["Gls"] = listPlayerDetail["Gls"].replace("-", 0)
listPlayerDetail["Gls"] = listPlayerDetail["Gls"].replace("", 0)
listPlayerDetail["Gls"] = listPlayerDetail["Gls"].astype(int)

listPlayerDetail["Asts"] = listPlayerDetail["Asts"].replace("-", 0)
listPlayerDetail["Asts"] = listPlayerDetail["Asts"].replace("", 0)
listPlayerDetail["Asts"] = listPlayerDetail["Asts"].astype(int)

listPlayerDetail["YC"] = listPlayerDetail["YC"].replace("-", 0)
listPlayerDetail["YC"] = listPlayerDetail["YC"].replace("", 0)
listPlayerDetail["YC"] = listPlayerDetail["YC"].astype(int)

listPlayerDetail["Second YC"] = listPlayerDetail["Second YC"].replace("-", 0)
listPlayerDetail["Second YC"] = listPlayerDetail["Second YC"].replace("", 0)
listPlayerDetail["Second YC"] = listPlayerDetail["Second YC"].astype(int)

listPlayerDetail["RC"] = listPlayerDetail["RC"].replace("-", 0)
listPlayerDetail["RC"] = listPlayerDetail["RC"].replace("", 0)
listPlayerDetail["RC"] = listPlayerDetail["RC"].astype(int)

listPlayerDetail["Subs On"] = listPlayerDetail["Subs On"].replace("-", 0)
listPlayerDetail["Subs On"] = listPlayerDetail["Subs On"].replace("", 0)
listPlayerDetail["Subs On"] = listPlayerDetail["Subs On"].astype(int)

listPlayerDetail["Min"] = listPlayerDetail["Min"].str.replace(".", "")
listPlayerDetail["Min"] = listPlayerDetail["Min"].str[:-1]
listPlayerDetail["Min"] = listPlayerDetail["Min"].replace("-", 0)
listPlayerDetail["Min"] = listPlayerDetail["Min"].replace("", 0)
listPlayerDetail["Min"] = listPlayerDetail["Min"].astype(int)

In [26]:
listPlayerDetail

,Player,Position,Age,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team,League
0,Kevin Müller,Goalkeeper,34,30,0,0,0,0,0,0,2700,1.FC Heidenheim 1846,Bundesliga
1,Frank Feller,Goalkeeper,21,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,Bundesliga
2,Vitus Eicher,Goalkeeper,34,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,Bundesliga
3,Paul Tschernuth,Goalkeeper,23,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,Bundesliga
4,Omar Traoré,Right-Back,27,26,0,1,5,0,0,0,2269,1.FC Heidenheim 1846,Bundesliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,Enso González,Left Winger,20,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,Premier League
3524,Chiquinho,Left Winger,25,1,0,0,0,0,0,1,6,Wolverhampton Wanderers,Premier League
3525,Nathan Fraser,Centre-Forward,20,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,Premier League
3526,Tom Edozie,Right Winger,18,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,Premier League


In [27]:
duplicatedListPlayerDetail = listPlayerDetail[listPlayerDetail.duplicated(subset=["Player", "Position", "Age", "League"], keep=False)]
duplicatedListPlayerDetail = duplicatedListPlayerDetail.sort_values(by="Player")
duplicatedListPlayerDetail

,Player,Position,Age,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team,League
2832,Aaron Ramsdale,Goalkeeper,26,25,0,0,2,0,0,0,2250,Southampton FC,Premier League
440,Aaron Ramsdale,Goalkeeper,26,0,0,0,0,0,0,0,0,Arsenal FC,Premier League
204,Alessandro Bianco,Central Midfield,22,1,0,0,0,0,0,1,17,ACF Fiorentina,Serie A
153,Alessandro Bianco,Central Midfield,22,29,1,0,8,0,0,5,2189,AC Monza,Serie A
3289,Alessandro Marcandalli,Centre-Back,22,6,0,0,2,0,0,1,317,Venezia FC,Serie A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2424,Urko González de Zárate,Defensive Midfield,24,11,0,1,1,0,0,1,868,RCD Espanyol Barcelona,LaLiga
2961,Warmed Omari,Centre-Back,25,0,0,0,0,0,0,0,0,Stade Rennais FC,Ligue 1
2154,Warmed Omari,Centre-Back,25,1,0,0,0,0,0,1,7,Olympique Lyon,Ligue 1
150,Warren Bondo,Central Midfield,21,20,0,0,3,1,0,1,1623,AC Monza,Serie A


In [28]:
listPlayerDetail.drop(
    listPlayerDetail[listPlayerDetail.duplicated(subset=["Player", "Position", "Age", "League"], keep=False)].index,
    inplace=True,
)
listPlayerDetail

,Player,Position,Age,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team,League
0,Kevin Müller,Goalkeeper,34,30,0,0,0,0,0,0,2700,1.FC Heidenheim 1846,Bundesliga
1,Frank Feller,Goalkeeper,21,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,Bundesliga
2,Vitus Eicher,Goalkeeper,34,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,Bundesliga
3,Paul Tschernuth,Goalkeeper,23,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,Bundesliga
4,Omar Traoré,Right-Back,27,26,0,1,5,0,0,0,2269,1.FC Heidenheim 1846,Bundesliga
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,Enso González,Left Winger,20,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,Premier League
3524,Chiquinho,Left Winger,25,1,0,0,0,0,0,1,6,Wolverhampton Wanderers,Premier League
3525,Nathan Fraser,Centre-Forward,20,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,Premier League
3526,Tom Edozie,Right Winger,18,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,Premier League


In [29]:
groupedDuplicatedListPlayerDetail = duplicatedListPlayerDetail.groupby(
    ["Player", "Position", "Age", "League"], as_index=False
).agg(
    {
        "MP": "sum",
        "Gls": "sum",
        "Asts": "sum",
        "YC": "sum",
        "Second YC": "sum",
        "RC": "sum",
        "Subs On": "sum",
        "Min": "sum",
        "Team": ":".join,
    }
)
groupedDuplicatedListPlayerDetailExtra = groupedDuplicatedListPlayerDetail["Team"].str.split(":", expand=True)

In [30]:
groupedDuplicatedListPlayerDetail = pd.concat([groupedDuplicatedListPlayerDetail, groupedDuplicatedListPlayerDetailExtra], axis=1)

In [31]:
del groupedDuplicatedListPlayerDetail["Team"]
groupedDuplicatedListPlayerDetail.rename(columns={0: "Team 1", 1: "Team 2"}, inplace=True)
groupedDuplicatedListPlayerDetail

,Player,Position,Age,League,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team 1,Team 2
0,Aaron Ramsdale,Goalkeeper,26,Premier League,25,0,0,2,0,0,0,2250,Southampton FC,Arsenal FC
1,Alessandro Bianco,Central Midfield,22,Serie A,30,1,0,8,0,0,6,2206,ACF Fiorentina,AC Monza
2,Alessandro Marcandalli,Centre-Back,22,Serie A,8,0,0,2,0,0,2,363,Venezia FC,Genoa CFC
3,Alessandro Vogliacco,Centre-Back,26,Serie A,21,2,0,5,0,0,4,1478,Parma Calcio 1913,Genoa CFC
4,Alessio Zerbin,Left Winger,26,Serie A,15,1,1,3,0,0,3,1079,Venezia FC,SSC Napoli
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Trevoh Chalobah,Centre-Back,25,Premier League,22,3,1,2,0,0,2,1719,Chelsea FC,Crystal Palace
100,Umar Sadiq,Centre-Forward,28,LaLiga,19,5,0,5,0,0,10,877,Real Sociedad,Valencia CF
101,Urko González de Zárate,Defensive Midfield,24,LaLiga,12,0,1,2,0,0,1,913,Real Sociedad,RCD Espanyol Barcelona
102,Warmed Omari,Centre-Back,25,Ligue 1,1,0,0,0,0,0,1,7,Stade Rennais FC,Olympique Lyon


In [32]:
listPlayerDetail.rename(columns={"Team": "Team 1"}, inplace=True)
listPlayerDetail["Team 2"] = ""
listPlayerDetail.insert(3, "League", listPlayerDetail.pop("League"))
listPlayerDetail

,Player,Position,Age,League,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team 1,Team 2
0,Kevin Müller,Goalkeeper,34,Bundesliga,30,0,0,0,0,0,0,2700,1.FC Heidenheim 1846,
1,Frank Feller,Goalkeeper,21,Bundesliga,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,
2,Vitus Eicher,Goalkeeper,34,Bundesliga,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,
3,Paul Tschernuth,Goalkeeper,23,Bundesliga,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,
4,Omar Traoré,Right-Back,27,Bundesliga,26,0,1,5,0,0,0,2269,1.FC Heidenheim 1846,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3523,Enso González,Left Winger,20,Premier League,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,
3524,Chiquinho,Left Winger,25,Premier League,1,0,0,0,0,0,1,6,Wolverhampton Wanderers,
3525,Nathan Fraser,Centre-Forward,20,Premier League,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,
3526,Tom Edozie,Right Winger,18,Premier League,0,0,0,0,0,0,0,0,Wolverhampton Wanderers,


In [33]:
listPlayerDetail = pd.concat([listPlayerDetail, groupedDuplicatedListPlayerDetail], axis=0)

In [34]:
listPlayerDetail["Team 2"] = listPlayerDetail["Team 2"].replace("", "-")
listPlayerDetail = listPlayerDetail.sort_values(by="Team 1")

In [35]:
listPlayerDetail = listPlayerDetail.reset_index(drop=True)
listPlayerDetail

,Player,Position,Age,League,MP,Gls,Asts,YC,Second YC,RC,Subs On,Min,Team 1,Team 2
0,Kevin Müller,Goalkeeper,34,Bundesliga,30,0,0,0,0,0,0,2700,1.FC Heidenheim 1846,-
1,Lennard Maloney,Defensive Midfield,25,Bundesliga,14,0,0,2,0,0,4,786,1.FC Heidenheim 1846,1.FSV Mainz 05
2,Niklas Dorsch,Defensive Midfield,27,Bundesliga,20,2,2,4,0,1,4,1183,1.FC Heidenheim 1846,FC Augsburg
3,Christopher Negele,Left Winger,20,Bundesliga,0,0,0,0,0,0,0,0,1.FC Heidenheim 1846,-
4,Stefan Schimmer,Centre-Forward,30,Bundesliga,13,0,0,1,0,0,13,237,1.FC Heidenheim 1846,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3419,André,Defensive Midfield,23,Premier League,28,0,0,7,0,0,2,2097,Wolverhampton Wanderers,-
3420,Jean-Ricner Bellegarde,Central Midfield,26,Premier League,30,2,6,3,0,0,12,1488,Wolverhampton Wanderers,-
3421,Marshall Munetsi,Central Midfield,28,Premier League,9,1,1,0,0,0,1,699,Wolverhampton Wanderers,-
3422,Tommy Doyle,Central Midfield,23,Premier League,24,0,1,1,0,0,21,476,Wolverhampton Wanderers,-


In [36]:
finalListPlayer["MP"] = ""
finalListPlayer["Gls"] = ""
finalListPlayer["Asts"] = ""
finalListPlayer["YC"] = ""
finalListPlayer["Second YC"] = ""
finalListPlayer["RC"] = ""
finalListPlayer["Subs On"] = ""
finalListPlayer["Min"] = ""
finalListPlayer["St"] = ""

for n in range(len(finalListPlayer["Player"])):
    for m in range(len(listPlayerDetail["Player"])):
        if ((finalListPlayer.loc[n, "Player"] == listPlayerDetail.loc[m, "Player"]) and (finalListPlayer.loc[n, "Position"] == listPlayerDetail.loc[m, "Position"]) and (finalListPlayer.loc[n, "Age"] == listPlayerDetail.loc[m, "Age"]) and ((finalListPlayer.loc[n, "Team"] == listPlayerDetail.loc[m, "Team 1"]) or (finalListPlayer.loc[n, "Team"] == listPlayerDetail.loc[m, "Team 2"]))):
            finalListPlayer.loc[n, "MP"] = listPlayerDetail.loc[m, "MP"]
            finalListPlayer.loc[n, "Gls"] = listPlayerDetail.loc[m, "Gls"]
            finalListPlayer.loc[n, "Asts"] = listPlayerDetail.loc[m, "Asts"]
            finalListPlayer.loc[n, "YC"] = listPlayerDetail.loc[m, "YC"]
            finalListPlayer.loc[n, "Second YC"] = listPlayerDetail.loc[m, "Second YC"]
            finalListPlayer.loc[n, "RC"] = listPlayerDetail.loc[m, "RC"]
            finalListPlayer.loc[n, "Subs On"] = listPlayerDetail.loc[m, "Subs On"]
            finalListPlayer.loc[n, "Min"] = listPlayerDetail.loc[m, "Min"]

In [37]:
uniqueValues = finalListPlayer["MP"].unique()
uniqueValues

array([np.int64(0), np.int64(5), np.int64(1), np.int64(13), np.int64(22),
       np.int64(15), np.int64(11), np.int64(10), np.int64(20),
       np.int64(3), np.int64(9), np.int64(28), np.int64(2), np.int64(7),
       np.int64(25), np.int64(21), np.int64(6), np.int64(18),
       np.int64(14), np.int64(29), np.int64(26), np.int64(16),
       np.int64(8), np.int64(12), np.int64(24), np.int64(4), np.int64(30),
       np.int64(27), np.int64(23), np.int64(17), np.int64(19),
       np.int64(32), np.int64(31), np.int64(33), np.int64(34)],
      dtype=object)

In [38]:
finalListPlayerFoot = finalListPlayer[finalListPlayer["Foot"].str.contains("\xa0")]
finalListPlayer.drop(
    finalListPlayer[finalListPlayer["Foot"].str.contains("\xa0")].index,
    inplace=True,
)
finalListPlayerFoot["Foot"] = finalListPlayerFoot["Foot"].str[:-10]

finalListPlayer = pd.concat([finalListPlayer, finalListPlayerFoot], axis=0)

/tmp/ipykernel_2016/1422069622.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalListPlayerFoot["Foot"] = finalListPlayerFoot["Foot"].str[:-10]


In [39]:
finalListPlayer["Position"] = finalListPlayer["Position"].replace("", "-")

finalListPlayer["Age"] = finalListPlayer["Age"].replace("-", 0)
finalListPlayer["Age"] = finalListPlayer["Age"].replace("", 0)
finalListPlayer["Age"] = finalListPlayer["Age"].astype(int)

finalListPlayer["Height"] = finalListPlayer["Height"].str.replace(",", "")
finalListPlayer["Height"] = finalListPlayer["Height"].str[:-1]
finalListPlayer["Height"] = finalListPlayer["Height"].replace("-", 0)
finalListPlayer["Height"] = finalListPlayer["Height"].replace("", 0)
finalListPlayer["Height"] = finalListPlayer["Height"].astype(int)

finalListPlayer["Foot"] = finalListPlayer["Foot"].replace("", "-")

finalListPlayer["Nation"] = finalListPlayer["Nation"].replace("", "-")

In [40]:
finalListPlayer["St"] = finalListPlayer["MP"].sub(finalListPlayer["Subs On"], fill_value=0)

In [41]:
uniquePositions = finalListPlayer["Position"].unique()
uniquePositions

array(['Goalkeeper', 'Centre-Back', 'Defensive Midfield', 'Left Winger',
       'Centre-Forward', 'Right-Back', 'Central Midfield', 'Left-Back',
       'Right Winger', 'Attacking Midfield', 'Right Midfield',
       'Left Midfield', 'Second Striker'], dtype=object)

In [42]:
finalListPlayer["Position"] = finalListPlayer["Position"].replace("Goalkeeper", "GK")
finalListPlayer["Position"] = finalListPlayer["Position"].replace(
    ["Centre-Back", "Left-Back", "Right-Back"], "DF"
)
finalListPlayer["Position"] = finalListPlayer["Position"].replace(
    [
        "Defensive Midfield",
        "Central Midfield",
        "Right Midfield",
        "Left Midfield",
        "Attacking Midfield",
    ],
    "MF",
)
finalListPlayer["Position"] = finalListPlayer["Position"].replace(
    ["Centre-Forward", "Left Winger", "Right Winger", "Second Striker"], "FW"
)

In [43]:
finalListPlayer["Mini Team Crest"] = ""
finalListPlayer["Big Team Crest"] = ""
finalListPlayer["League"] = ""
finalListPlayer["Team Link"] = ""
finalListPlayer["Team Link 2"] = ""

for p in range(len(finalListPlayer["Team"])):
    for r in range(len(finalListClub["Team"])):
        if finalListPlayer.loc[p, "Team"] == finalListClub.loc[r, "Team"]:
            finalListPlayer.loc[p, "Mini Team Crest"] = finalListClub.loc[r, "Mini Team Crest"]
            finalListPlayer.loc[p, "Big Team Crest"] = finalListClub.loc[r, "Big Team Crest"]
            finalListPlayer.loc[p, "League"] = finalListClub.loc[r, "League"]
            finalListPlayer.loc[p, "Team Link"] = finalListClub.loc[r, "Team Link"]
            finalListPlayer.loc[p, "Team Link 2"] = finalListClub.loc[r, "Team Link 2"]

In [44]:
finalListPlayer = finalListPlayer.sort_values(by="Team")
finalListPlayer = finalListPlayer.reset_index(drop=True)

In [45]:
finalListPlayer.insert(10, "St", finalListPlayer.pop("St"))
finalListPlayer.insert(7, "League", finalListPlayer.pop("League"))

In [46]:
finalListClub.insert(10, "Pts/MP", finalListClub.pop("Pts/MP"))
finalListClub.insert(12, "Team Link", finalListClub.pop("Team Link"))
finalListClub.insert(13, "Mini Team Crest", finalListClub.pop("Mini Team Crest"))
finalListClub.insert(13, "Big Team Crest", finalListClub.pop("Big Team Crest"))

finalListClub = finalListClub.sort_values(by="Pts/MP", ascending=False)
finalListClub = finalListClub.reset_index(drop=True)

In [47]:
response = req.get(
    "https://flagpedia.net/index",
    headers=headers,
)
soup = bs(response.content, "html.parser")
table = soup.find("ul", class_="flag-grid")
flagList = pd.DataFrame(
    {
        "Flag": [(i.text) for i in table.select("li a")],
        "Flag Link": [("https://flagpedia.net" + i["href"]) for i in table.select("li a")],
    }
)

In [48]:
flagList["Flag"] = flagList["Flag"].str.replace("\n\n", "")
flagList["Flag"] = flagList["Flag"].str.replace("\n", "")

In [49]:
flagListCode = pd.DataFrame()
for nationCode in flagList["Flag Link"]:
    responseSecond = req.get(
        nationCode,
        headers=headers,
    )
    soupSecond = bs(responseSecond.content, "html.parser")
    tableSecond = soupSecond.find("table", class_="table-dl")
    flagCode = pd.DataFrame(
        {
            "Flag Code": [(i.text) for i in tableSecond.select("tbody tr:nth-of-type(2) td")],
        }
    )

    flagListCode = pd.concat([flagListCode, flagCode], axis=0)

In [50]:
flagListCode = flagListCode.reset_index(drop=True)
flagListCodeOther = flagListCode["Flag Code"].str.split(",", expand=True)

In [51]:
flagListCode = pd.concat([flagListCode, flagListCodeOther], axis=1)

In [52]:
flagListCode.rename(columns={0: "Flag Code 1", 1: "Flag Code 2"}, inplace=True)
flagListCode["Flag Code 2"] = flagListCode["Flag Code 2"].str[:-13]

In [53]:
flagList = pd.concat([flagList, flagListCode], axis=1)

In [54]:
flagListFiltered = flagList[flagList["Flag Code 1"].str.contains("(ISO 3166-2:GB)")]
flagList.drop(
    flagList[flagList["Flag Code 1"].str.contains("(ISO 3166-2:GB)")].index,
    inplace=True,
)

/tmp/ipykernel_2016/453612881.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  flagListFiltered = flagList[flagList["Flag Code 1"].str.contains("(ISO 3166-2:GB)")]
/tmp/ipykernel_2016/453612881.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  flagList[flagList["Flag Code 1"].str.contains("(ISO 3166-2:GB)")].index,


In [55]:
flagListFiltered["Flag Code 1"] = flagListFiltered["Flag Code 1"].str[:-16]
flagListFiltered

/tmp/ipykernel_2016/693478608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flagListFiltered["Flag Code 1"] = flagListFiltered["Flag Code 1"].str[:-16]


,Flag,Flag Link,Flag Code,Flag Code 1,Flag Code 2
67,England,https://flagpedia.net/england,GB-ENG (ISO 3166-2:GB),GB-ENG,None
166,Northern Ireland,https://flagpedia.net/northern-ireland,GB-NIR (ISO 3166-2:GB),GB-NIR,None
198,Scotland,https://flagpedia.net/scotland,GB-SCT (ISO 3166-2:GB),GB-SCT,None
248,Wales,https://flagpedia.net/wales,GB-WLS (ISO 3166-2:GB),GB-WLS,None


In [56]:
flagList = pd.concat([flagList, flagListFiltered], axis=0)
flagList = flagList.reset_index(drop=True)

In [57]:
nationCodes = [
    "GB-ENG",
    "ES",
    "FR",
    "DE",
    "IT",
]

for l in range(len(finalListClub["League"])):
    if finalListClub.loc[l, "League"] == "Premier League":
        finalListClub.loc[l, "Nation Flag Code"] = "GB-ENG"
    elif finalListClub.loc[l, "League"] == "LaLiga":
        finalListClub.loc[l, "Nation Flag Code"] = "ES"
    elif finalListClub.loc[l, "League"] == "Ligue 1":
        finalListClub.loc[l, "Nation Flag Code"] = "FR"
    elif finalListClub.loc[l, "League"] == "Bundesliga":
        finalListClub.loc[l, "Nation Flag Code"] = "DE"
    elif finalListClub.loc[l, "League"] == "Serie A":
        finalListClub.loc[l, "Nation Flag Code"] = "IT"

finalListClub

,Ranking,Team,MP,W,D,L,Goals,Pts/MP,+/-,Pts,Team Link,League,Mini Team Crest,Big Team Crest,Team Link 2,Nation Flag Code
0,1,Paris Saint-Germain,30,24,6,0,83:28,2.60,55,78,https://www.transfermarkt.com/fc-paris-saint-g...,Ligue 1,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-paris-saint-g...,FR
1,1,Bayern Munich,30,22,6,2,87:29,2.40,58,72,https://www.transfermarkt.com/fc-bayern-munche...,Bundesliga,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-bayern-munche...,DE
2,1,Liverpool FC,33,24,7,2,75:31,2.39,44,79,https://www.transfermarkt.com/fc-liverpool/kad...,Premier League,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-liverpool/lei...,GB-ENG
3,1,FC Barcelona,33,24,4,5,89:32,2.30,57,76,https://www.transfermarkt.com/fc-barcelona/kad...,LaLiga,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-barcelona/lei...,ES
4,2,Real Madrid,32,21,6,5,65:31,2.16,34,69,https://www.transfermarkt.com/real-madrid/kade...,LaLiga,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/leis...,ES
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,19,Leicester City,33,4,6,23,27:73,0.55,-46,18,https://www.transfermarkt.com/leicester-city/k...,Premier League,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/leicester-city/l...,GB-ENG
92,20,Real Valladolid CF,32,4,4,24,23:76,0.50,-53,16,https://www.transfermarkt.com/real-valladolid/...,LaLiga,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-valladolid/...,ES
93,18,Montpellier HSC,30,4,3,23,22:71,0.50,-49,15,https://www.transfermarkt.com/montpellier-hsc/...,Ligue 1,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/montpellier-hsc/...,FR
94,20,AC Monza,33,2,9,22,25:57,0.45,-32,15,https://www.transfermarkt.com/ac-monza/kader/v...,Serie A,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/ac-monza/leistun...,IT


In [58]:
for n in range(len(finalListPlayer["Nation"])):
    for f in range(len(flagList["Flag"])):
        if finalListPlayer.loc[n, "Nation"] == flagList.loc[f, "Flag"]:
            finalListPlayer.loc[n, "Flag Code"] = flagList.loc[f, "Flag Code 1"]

In [59]:
finalListPlayerFiltered = finalListPlayer[finalListPlayer["Flag Code"].isna()]
finalListPlayer.drop(
    finalListPlayer[finalListPlayer["Flag Code"].isna()].index,
    inplace=True,
)
finalListPlayerFiltered[["Player", "Nation", "Flag Code"]]

,Player,Nation,Flag Code
37,Woo-yeong Jeong,"Korea, South",NaN
61,Jae-sung Lee,"Korea, South",NaN
62,Hyun-seok Hong,"Korea, South",NaN
114,Silvere Ganvoula,Congo,NaN
124,Jean-Daniel Akpa Akpro,Cote d'Ivoire,NaN
...,...,...,...
2519,Nicolas Pépé,Cote d'Ivoire,NaN
2531,Vladimír Coufal,Czech Republic,NaN
2547,Tomas Soucek,Czech Republic,NaN
2552,Hee-chan Hwang,"Korea, South",NaN


In [60]:
uniqueNations = finalListPlayerFiltered["Nation"].unique()
uniqueNations

array(['Korea, South', 'Congo', "Cote d'Ivoire", 'Türkiye',
       'Bosnia-Herzegovina', 'Czech Republic', 'The Gambia'], dtype=object)

In [61]:
finalListPlayerFilteredCongo = finalListPlayerFiltered[finalListPlayerFiltered["Nation"] == "Congo"]
finalListPlayerFilteredCongo

,Player,Position,Age,Height,Foot,Nation,Nation Flag Link,League,Team,Market Value,...,YC,Second YC,RC,Subs On,Min,Mini Team Crest,Big Team Crest,Team Link,Team Link 2,Flag Code
114,Silvere Ganvoula,FW,28,191,right,Congo,https://tmssl.akamaized.net//images/flagge/ver...,Serie A,AC Monza,2.0,...,0,0,0,3,325,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/ac-monza/kader/v...,https://www.transfermarkt.com/ac-monza/leistun...,NaN
320,Melvin Zinga,GK,23,187,left,Congo,https://tmssl.akamaized.net//images/flagge/ver...,Ligue 1,Angers SCO,0.2,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/sco-angers/kader...,https://www.transfermarkt.com/sco-angers/leist...,NaN
682,Antoine Makoumbou,MF,26,187,right,Congo,https://tmssl.akamaized.net//images/flagge/ver...,Serie A,Cagliari Calcio,4.0,...,1,1,0,5,1772,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/cagliari-calcio/...,https://www.transfermarkt.com/cagliari-calcio/...,NaN
1649,Gabriel Charpentier,FW,25,188,right,Congo,https://tmssl.akamaized.net//images/flagge/ver...,Serie A,Parma Calcio 1913,1.6,...,1,0,0,10,264,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/parma-calcio-191...,https://www.transfermarkt.com/parma-calcio-191...,NaN


In [62]:
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Korea, South", "South Korea")
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Cote d'Ivoire", "Côte d'Ivoire")
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Bosnia-Herzegovina", "Bosnia and Herzegovina")
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Türkiye", "Turkey")
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Congo", "Republic of the Congo")
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Czech Republic", "Czechia")
finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("The Gambia", "Gambia")

/tmp/ipykernel_2016/2693570354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Korea, South", "South Korea")
/tmp/ipykernel_2016/2693570354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  finalListPlayerFiltered["Nation"] = finalListPlayerFiltered["Nation"].str.replace("Cote d'Ivoire", "Côte d'Ivoire")
/tmp/ipykernel_2016/2693570354.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [63]:
finalListPlayerFiltered = finalListPlayerFiltered.reset_index(drop=True)

for n in range(len(finalListPlayerFiltered["Nation"])):
    for f in range(len(flagList["Flag"])):
        if finalListPlayerFiltered.loc[n, "Nation"] == flagList.loc[f, "Flag"]:
            finalListPlayerFiltered.loc[n, "Flag Code"] = flagList.loc[f, "Flag Code 1"]

finalListPlayerFiltered

,Player,Position,Age,Height,Foot,Nation,Nation Flag Link,League,Team,Market Value,...,YC,Second YC,RC,Subs On,Min,Mini Team Crest,Big Team Crest,Team Link,Team Link 2,Flag Code
0,Woo-yeong Jeong,MF,25,179,right,South Korea,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,1.FC Union Berlin,4.0,...,2,0,0,6,1272,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/1-fc-union-berli...,https://www.transfermarkt.com/1-fc-union-berli...,KR
1,Jae-sung Lee,MF,32,180,left,South Korea,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,1.FSV Mainz 05,2.5,...,4,0,0,2,2347,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/1-fsv-mainz-05/k...,https://www.transfermarkt.com/1-fsv-mainz-05/l...,KR
2,Hyun-seok Hong,MF,25,177,left,South Korea,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,1.FSV Mainz 05,3.5,...,0,0,0,16,480,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/1-fsv-mainz-05/k...,https://www.transfermarkt.com/1-fsv-mainz-05/l...,KR
3,Silvere Ganvoula,FW,28,191,right,Republic of the Congo,https://tmssl.akamaized.net//images/flagge/ver...,Serie A,AC Monza,2.0,...,0,0,0,3,325,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/ac-monza/kader/v...,https://www.transfermarkt.com/ac-monza/leistun...,CG
4,Jean-Daniel Akpa Akpro,MF,32,180,right,Côte d'Ivoire,https://tmssl.akamaized.net//images/flagge/ver...,Serie A,AC Monza,0.5,...,3,0,0,2,548,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/ac-monza/kader/v...,https://www.transfermarkt.com/ac-monza/leistun...,CI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Nicolas Pépé,FW,29,183,left,Côte d'Ivoire,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Villarreal CF,6.0,...,1,0,0,12,952,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-villarreal/ka...,https://www.transfermarkt.com/fc-villarreal/le...,CI
100,Vladimír Coufal,DF,32,174,right,Czechia,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,West Ham United,7.0,...,4,0,0,8,856,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/west-ham-united/...,https://www.transfermarkt.com/west-ham-united/...,CZ
101,Tomas Soucek,MF,30,192,right,Czechia,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,West Ham United,22.0,...,8,0,0,5,2163,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/west-ham-united/...,https://www.transfermarkt.com/west-ham-united/...,CZ
102,Hee-chan Hwang,FW,29,177,right,South Korea,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Wolverhampton Wanderers,17.0,...,0,0,0,14,635,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/wolverhampton-wa...,https://www.transfermarkt.com/wolverhampton-wa...,KR


In [64]:
finalListPlayer = pd.concat([finalListPlayer, finalListPlayerFiltered], axis=0)

finalListPlayer = finalListPlayer.sort_values(by="Market Value", ascending=False)
finalListPlayer = finalListPlayer.reset_index(drop=True)

finalListPlayer

,Player,Position,Age,Height,Foot,Nation,Nation Flag Link,League,Team,Market Value,...,YC,Second YC,RC,Subs On,Min,Mini Team Crest,Big Team Crest,Team Link,Team Link 2,Flag Code
0,Erling Haaland,FW,24,195,left,Norway,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Manchester City,200.0,...,2,0,0,0,2485,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/manchester-city/...,https://www.transfermarkt.com/manchester-city/...,NO
1,Vinicius Junior,FW,24,176,right,Brazil,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,200.0,...,8,0,1,5,1972,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,BR
2,Lamine Yamal,FW,17,180,left,Spain,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,FC Barcelona,180.0,...,3,0,0,3,2452,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-barcelona/kad...,https://www.transfermarkt.com/fc-barcelona/lei...,ES
3,Jude Bellingham,MF,21,186,right,England,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,180.0,...,3,0,1,1,2107,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,GB-ENG
4,Kylian Mbappé,FW,26,178,right,France,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,170.0,...,3,0,1,0,2467,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,Tom Negrel,GK,22,192,right,France,https://tmssl.akamaized.net//images/flagge/ver...,Ligue 1,AJ Auxerre,0.0,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/aj-auxerre/kader...,https://www.transfermarkt.com/aj-auxerre/leist...,FR
2576,Luca Novodomsky,GK,19,0,-,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,Bayer 04 Leverkusen,0.0,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/bayer-04-leverku...,https://www.transfermarkt.com/bayer-04-leverku...,DE
2577,Niklas Swider,MF,18,188,right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,Borussia Mönchengladbach,0.0,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/borussia-monchen...,https://www.transfermarkt.com/borussia-monchen...,DE
2578,Harry Amass,DF,18,181,left,England,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Manchester United,0.0,...,0,0,0,1,166,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/manchester-unite...,https://www.transfermarkt.com/manchester-unite...,GB-ENG


In [65]:
finalListClub["Id"] = range(1, len(finalListClub) + 1)
finalListClub.insert(0, "Id", finalListClub.pop("Id"))

finalListPlayer["Id"] = range(1, len(finalListPlayer) + 1)
finalListPlayer.insert(0, "Id", finalListPlayer.pop("Id"))

In [66]:
finalListPlayer.rename(
    columns={"Position": "Pos", "Market Value": "Value", "Gls": "Gl", "Asts": "As", "Second YC": "SYC"}, inplace=True
)

In [67]:
finalListPlayer

,Id,Player,Pos,Age,Height,Foot,Nation,Nation Flag Link,League,Team,...,YC,SYC,RC,Subs On,Min,Mini Team Crest,Big Team Crest,Team Link,Team Link 2,Flag Code
0,1,Erling Haaland,FW,24,195,left,Norway,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Manchester City,...,2,0,0,0,2485,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/manchester-city/...,https://www.transfermarkt.com/manchester-city/...,NO
1,2,Vinicius Junior,FW,24,176,right,Brazil,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,...,8,0,1,5,1972,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,BR
2,3,Lamine Yamal,FW,17,180,left,Spain,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,FC Barcelona,...,3,0,0,3,2452,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-barcelona/kad...,https://www.transfermarkt.com/fc-barcelona/lei...,ES
3,4,Jude Bellingham,MF,21,186,right,England,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,...,3,0,1,1,2107,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,GB-ENG
4,5,Kylian Mbappé,FW,26,178,right,France,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,...,3,0,1,0,2467,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,FR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,2576,Tom Negrel,GK,22,192,right,France,https://tmssl.akamaized.net//images/flagge/ver...,Ligue 1,AJ Auxerre,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/aj-auxerre/kader...,https://www.transfermarkt.com/aj-auxerre/leist...,FR
2576,2577,Luca Novodomsky,GK,19,0,-,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,Bayer 04 Leverkusen,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/bayer-04-leverku...,https://www.transfermarkt.com/bayer-04-leverku...,DE
2577,2578,Niklas Swider,MF,18,188,right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,Borussia Mönchengladbach,...,0,0,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/borussia-monchen...,https://www.transfermarkt.com/borussia-monchen...,DE
2578,2579,Harry Amass,DF,18,181,left,England,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Manchester United,...,0,0,0,1,166,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/manchester-unite...,https://www.transfermarkt.com/manchester-unite...,GB-ENG


In [68]:
finalListPlayer["YC"] = finalListPlayer["YC"] + (finalListPlayer["SYC"] * 2)
finalListPlayer["RC"] = finalListPlayer["RC"] + finalListPlayer["SYC"]

In [69]:
del finalListPlayer["SYC"]

In [70]:
finalListPlayer.rename(columns={"Gl": "G", "As": "A"}, inplace=True)
finalListPlayer["G+A"] = finalListPlayer["G"] + finalListPlayer["A"]
finalListPlayer.insert(15, "G+A", finalListPlayer.pop("G+A"))

In [71]:
duplicatedFinalListPlayer = finalListPlayer[finalListPlayer.duplicated(subset=["Player", "Age"], keep=False)]
duplicatedFinalListPlayer = duplicatedFinalListPlayer.sort_values(by="Player")
duplicatedFinalListPlayer

,Id,Player,Pos,Age,Height,Foot,Nation,Nation Flag Link,League,Team,...,G+A,YC,RC,Subs On,Min,Mini Team Crest,Big Team Crest,Team Link,Team Link 2,Flag Code
57,58,Vitinha,MF,25,172,right,Portugal,https://tmssl.akamaized.net//images/flagge/ver...,Ligue 1,Paris Saint-Germain,...,6,1,0,9,1855,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-paris-saint-g...,https://www.transfermarkt.com/fc-paris-saint-g...,PT
913,914,Vitinha,FW,25,178,right,Portugal,https://tmssl.akamaized.net//images/flagge/ver...,Serie A,Genoa CFC,...,2,1,0,7,1011,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/genua-cfc/kader/...,https://www.transfermarkt.com/genua-cfc/leistu...,PT


In [72]:
goalsAgainstandFor = finalListClub["Goals"].str.split(":", expand=True)
finalListClub = pd.concat([finalListClub, goalsAgainstandFor], axis=1)
finalListClub.rename(columns={0: "Goals For", 1: "Goals Against"}, inplace=True)

In [73]:
finalListClub["League Ranking"] = finalListClub["Ranking"].str[:-2]
finalListClub["League Ranking"].values

array(['1', '1', '1', '1', '2', '2', '1', '2', '2', '3', '3', '5', '2',
       '4', '4', '3', '5', '3', '4', '4', '7', '6', '3', '6', '6', '5',
       '7', '7', '5', '8', '4', '5', '6', '6', '9', '7', '8', '8', '8',
       '9', '9', '9', '10', '10', '11', '11', '8', '12', '7', '9', '12',
       '11', '10', '11', '10', '11', '10', '13', '13', '12', '12', '14',
       '13', '15', '14', '13', '15', '12', '16', '17', '13', '16', '14',
       '14', '17', '15', '15', '18', '14', '15', '19', '16', '17', '16',
       '17', '19', '18', '16', '17', '18', '18', '19', '20', '18', '20',
       '20'], dtype=object)

In [74]:
del finalListClub["Ranking"]
del finalListClub["Goals"]

In [75]:
finalListClub["League Ranking"] = finalListClub["League Ranking"].astype(int)
finalListClub["Goals For"] = finalListClub["Goals For"].astype(int)
finalListClub["Goals Against"] = finalListClub["Goals Against"].astype(int)
finalListClub["MP"] = finalListClub["MP"].astype(int)
finalListClub["W"] = finalListClub["W"].astype(int)
finalListClub["D"] = finalListClub["D"].astype(int)
finalListClub["L"] = finalListClub["L"].astype(int)
finalListClub["+/-"] = finalListClub["+/-"].astype(int)
finalListClub["Pts"] = finalListClub["Pts"].astype(int)

In [76]:
finalListClub = finalListClub.sort_values(
    by=["Pts/MP", "MP", "Pts", "+/-"], ascending=[False, False, False, False]
)
finalListClub = finalListClub.reset_index(drop=True)
finalListClub["Id"] = range(1, len(finalListClub) + 1)

In [77]:
finalListClub.insert(8, "Pts/MP", finalListClub.pop("Pts/MP"))
finalListClub.insert(2, "League", finalListClub.pop("League"))
finalListClub.insert(11, "Team Link 2", finalListClub.pop("Team Link 2"))
finalListClub.insert(3, "Nation Flag Code", finalListClub.pop("Nation Flag Code"))
finalListClub.insert(8, "Goals For", finalListClub.pop("Goals For"))
finalListClub.insert(9, "Goals Against", finalListClub.pop("Goals Against"))
finalListClub.insert(1, "League Ranking", finalListClub.pop("League Ranking"))
finalListClub

,Id,League Ranking,Team,League,Nation Flag Code,MP,W,D,L,Goals For,Goals Against,+/-,Pts,Pts/MP,Team Link,Team Link 2,Mini Team Crest,Big Team Crest
0,1,1,Paris Saint-Germain,Ligue 1,FR,30,24,6,0,83,28,55,78,2.60,https://www.transfermarkt.com/fc-paris-saint-g...,https://www.transfermarkt.com/fc-paris-saint-g...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
1,2,1,Bayern Munich,Bundesliga,DE,30,22,6,2,87,29,58,72,2.40,https://www.transfermarkt.com/fc-bayern-munche...,https://www.transfermarkt.com/fc-bayern-munche...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
2,3,1,Liverpool FC,Premier League,GB-ENG,33,24,7,2,75,31,44,79,2.39,https://www.transfermarkt.com/fc-liverpool/kad...,https://www.transfermarkt.com/fc-liverpool/lei...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
3,4,1,FC Barcelona,LaLiga,ES,33,24,4,5,89,32,57,76,2.30,https://www.transfermarkt.com/fc-barcelona/kad...,https://www.transfermarkt.com/fc-barcelona/lei...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
4,5,2,Real Madrid,LaLiga,ES,32,21,6,5,65,31,34,69,2.16,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,92,19,Leicester City,Premier League,GB-ENG,33,4,6,23,27,73,-46,18,0.55,https://www.transfermarkt.com/leicester-city/k...,https://www.transfermarkt.com/leicester-city/l...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
92,93,20,Real Valladolid CF,LaLiga,ES,32,4,4,24,23,76,-53,16,0.50,https://www.transfermarkt.com/real-valladolid/...,https://www.transfermarkt.com/real-valladolid/...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
93,94,18,Montpellier HSC,Ligue 1,FR,30,4,3,23,22,71,-49,15,0.50,https://www.transfermarkt.com/montpellier-hsc/...,https://www.transfermarkt.com/montpellier-hsc/...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
94,95,20,AC Monza,Serie A,IT,33,2,9,22,25,57,-32,15,0.45,https://www.transfermarkt.com/ac-monza/kader/v...,https://www.transfermarkt.com/ac-monza/leistun...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...


In [78]:
finalListPlayer["League Ranking All"] = 0
finalListPlayer["League Ranking"] = 0

for p in range(len(finalListPlayer["Team"])):
    for r in range(len(finalListClub["Team"])):
        if finalListPlayer.loc[p, "Team"] == finalListClub.loc[r, "Team"]:
            finalListPlayer.loc[p, "League Ranking All"] = finalListClub.loc[r, "Id"]
            finalListPlayer.loc[p, "League Ranking"] = finalListClub.loc[r, "League Ranking"]
finalListPlayer

,Id,Player,Pos,Age,Height,Foot,Nation,Nation Flag Link,League,Team,...,RC,Subs On,Min,Mini Team Crest,Big Team Crest,Team Link,Team Link 2,Flag Code,League Ranking All,League Ranking
0,1,Erling Haaland,FW,24,195,left,Norway,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Manchester City,...,0,0,2485,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/manchester-city/...,https://www.transfermarkt.com/manchester-city/...,NO,17,3
1,2,Vinicius Junior,FW,24,176,right,Brazil,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,...,1,5,1972,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,BR,5,2
2,3,Lamine Yamal,FW,17,180,left,Spain,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,FC Barcelona,...,0,3,2452,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/fc-barcelona/kad...,https://www.transfermarkt.com/fc-barcelona/lei...,ES,4,1
3,4,Jude Bellingham,MF,21,186,right,England,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,...,1,1,2107,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,GB-ENG,5,2
4,5,Kylian Mbappé,FW,26,178,right,France,https://tmssl.akamaized.net//images/flagge/ver...,LaLiga,Real Madrid,...,1,0,2467,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/real-madrid/kade...,https://www.transfermarkt.com/real-madrid/leis...,FR,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2575,2576,Tom Negrel,GK,22,192,right,France,https://tmssl.akamaized.net//images/flagge/ver...,Ligue 1,AJ Auxerre,...,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/aj-auxerre/kader...,https://www.transfermarkt.com/aj-auxerre/leist...,FR,55,11
2576,2577,Luca Novodomsky,GK,19,0,-,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,Bayer 04 Leverkusen,...,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/bayer-04-leverku...,https://www.transfermarkt.com/bayer-04-leverku...,DE,8,2
2577,2578,Niklas Swider,MF,18,188,right,Germany,https://tmssl.akamaized.net//images/flagge/ver...,Bundesliga,Borussia Mönchengladbach,...,0,0,0,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/borussia-monchen...,https://www.transfermarkt.com/borussia-monchen...,DE,40,9
2578,2579,Harry Amass,DF,18,181,left,England,https://tmssl.akamaized.net//images/flagge/ver...,Premier League,Manchester United,...,0,1,166,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...,https://www.transfermarkt.com/manchester-unite...,https://www.transfermarkt.com/manchester-unite...,GB-ENG,66,14


In [79]:
finalListClubSecond = finalListClub.copy()
finalListClubSecond = finalListClubSecond.sort_values(by="League Ranking")
finalListClubSecond = finalListClubSecond.reset_index(drop=True)
finalListClubSecond

,Id,League Ranking,Team,League,Nation Flag Code,MP,W,D,L,Goals For,Goals Against,+/-,Pts,Pts/MP,Team Link,Team Link 2,Mini Team Crest,Big Team Crest
0,1,1,Paris Saint-Germain,Ligue 1,FR,30,24,6,0,83,28,55,78,2.60,https://www.transfermarkt.com/fc-paris-saint-g...,https://www.transfermarkt.com/fc-paris-saint-g...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
1,2,1,Bayern Munich,Bundesliga,DE,30,22,6,2,87,29,58,72,2.40,https://www.transfermarkt.com/fc-bayern-munche...,https://www.transfermarkt.com/fc-bayern-munche...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
2,3,1,Liverpool FC,Premier League,GB-ENG,33,24,7,2,75,31,44,79,2.39,https://www.transfermarkt.com/fc-liverpool/kad...,https://www.transfermarkt.com/fc-liverpool/lei...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
3,4,1,FC Barcelona,LaLiga,ES,33,24,4,5,89,32,57,76,2.30,https://www.transfermarkt.com/fc-barcelona/kad...,https://www.transfermarkt.com/fc-barcelona/lei...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
4,6,1,Inter Milan,Serie A,IT,33,21,8,4,72,32,40,71,2.15,https://www.transfermarkt.com/inter-mailand/ka...,https://www.transfermarkt.com/inter-mailand/le...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,81,19,CD Leganés,LaLiga,ES,32,6,11,15,29,48,-19,29,0.91,https://www.transfermarkt.com/cd-leganes/kader...,https://www.transfermarkt.com/cd-leganes/leist...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
92,92,19,Leicester City,Premier League,GB-ENG,33,4,6,23,27,73,-46,18,0.55,https://www.transfermarkt.com/leicester-city/k...,https://www.transfermarkt.com/leicester-city/l...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
93,93,20,Real Valladolid CF,LaLiga,ES,32,4,4,24,23,76,-53,16,0.50,https://www.transfermarkt.com/real-valladolid/...,https://www.transfermarkt.com/real-valladolid/...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...
94,95,20,AC Monza,Serie A,IT,33,2,9,22,25,57,-32,15,0.45,https://www.transfermarkt.com/ac-monza/kader/v...,https://www.transfermarkt.com/ac-monza/leistun...,https://tmssl.akamaized.net//images/wappen/tin...,https://tmssl.akamaized.net//images/wappen/hea...


In [80]:
finalListPlayerLigue1 = finalListPlayer.copy()
finalListPlayerBundesliga = finalListPlayer.copy()
finalListPlayerPremierLeague = finalListPlayer.copy()
finalListPlayerSerieA = finalListPlayer.copy()
finalListPlayerLaLiga = finalListPlayer.copy()

finalListPlayerLigue1 = finalListPlayerLigue1[finalListPlayerLigue1["League"] == "Ligue 1"]
finalListPlayerBundesliga = finalListPlayerBundesliga[finalListPlayerBundesliga["League"] == "Bundesliga"]
finalListPlayerPremierLeague = finalListPlayerPremierLeague[finalListPlayerPremierLeague["League"] == "Premier League"]
finalListPlayerSerieA = finalListPlayerSerieA[finalListPlayerSerieA["League"] == "Serie A"]
finalListPlayerLaLiga = finalListPlayerLaLiga[finalListPlayerLaLiga["League"] == "LaLiga"]

finalListPlayerLigue1["Id"] = range(1, len(finalListPlayerLigue1) + 1)
finalListPlayerLigue1 = finalListPlayerLigue1.reset_index(drop=True)
del finalListPlayerLigue1["League Ranking All"]
finalListPlayerLigue1.rename(columns={"League Ranking": "TR"}, inplace=True)

finalListPlayerBundesliga["Id"] = range(1, len(finalListPlayerBundesliga) + 1)
finalListPlayerBundesliga = finalListPlayerBundesliga.reset_index(drop=True)
del finalListPlayerBundesliga["League Ranking All"]
finalListPlayerBundesliga.rename(columns={"League Ranking": "TR"}, inplace=True)

finalListPlayerPremierLeague["Id"] = range(1, len(finalListPlayerPremierLeague) + 1)
finalListPlayerPremierLeague = finalListPlayerPremierLeague.reset_index(drop=True)
del finalListPlayerPremierLeague["League Ranking All"]
finalListPlayerPremierLeague.rename(columns={"League Ranking": "TR"}, inplace=True)

finalListPlayerSerieA["Id"] = range(1, len(finalListPlayerSerieA) + 1)
finalListPlayerSerieA = finalListPlayerSerieA.reset_index(drop=True)
del finalListPlayerSerieA["League Ranking All"]
finalListPlayerSerieA.rename(columns={"League Ranking": "TR"}, inplace=True)

finalListPlayerLaLiga["Id"] = range(1, len(finalListPlayerLaLiga) + 1)
finalListPlayerLaLiga = finalListPlayerLaLiga.reset_index(drop=True)
del finalListPlayerLaLiga["League Ranking All"]
finalListPlayerLaLiga.rename(columns={"League Ranking": "TR"}, inplace=True)

In [81]:
del finalListPlayer["League Ranking"]
finalListPlayer.rename(columns={"League Ranking All": "TR"}, inplace=True)

In [82]:
nanCountsPlayer = finalListPlayer.isna().sum()
print(nanCountsPlayer)

Id                  0
Player              0
Pos                 0
Age                 0
Height              0
Foot                0
Nation              0
Nation Flag Link    0
League              0
Team                0
Value               0
MP                  0
St                  0
G                   0
A                   0
G+A                 0
YC                  0
RC                  0
Subs On             0
Min                 0
Mini Team Crest     0
Big Team Crest      0
Team Link           0
Team Link 2         0
Flag Code           0
TR                  0
dtype: int64


In [83]:
nanCountsClub = finalListClub.isna().sum()
print(nanCountsClub)

Id                  0
League Ranking      0
Team                0
League              0
Nation Flag Code    0
MP                  0
W                   0
D                   0
L                   0
Goals For           0
Goals Against       0
+/-                 0
Pts                 0
Pts/MP              0
Team Link           0
Team Link 2         0
Mini Team Crest     0
Big Team Crest      0
dtype: int64


In [84]:
nanCountsClubSecond = finalListClubSecond.isna().sum()
print(nanCountsClubSecond)

Id                  0
League Ranking      0
Team                0
League              0
Nation Flag Code    0
MP                  0
W                   0
D                   0
L                   0
Goals For           0
Goals Against       0
+/-                 0
Pts                 0
Pts/MP              0
Team Link           0
Team Link 2         0
Mini Team Crest     0
Big Team Crest      0
dtype: int64


In [85]:
nanCountsPlayerLigue1 = finalListPlayerLigue1.isna().sum()
print(nanCountsPlayerLigue1)

Id                  0
Player              0
Pos                 0
Age                 0
Height              0
Foot                0
Nation              0
Nation Flag Link    0
League              0
Team                0
Value               0
MP                  0
St                  0
G                   0
A                   0
G+A                 0
YC                  0
RC                  0
Subs On             0
Min                 0
Mini Team Crest     0
Big Team Crest      0
Team Link           0
Team Link 2         0
Flag Code           0
TR                  0
dtype: int64


In [86]:
nanCountsPlayerBundesliga = finalListPlayerBundesliga.isna().sum()
print(nanCountsPlayerBundesliga)

Id                  0
Player              0
Pos                 0
Age                 0
Height              0
Foot                0
Nation              0
Nation Flag Link    0
League              0
Team                0
Value               0
MP                  0
St                  0
G                   0
A                   0
G+A                 0
YC                  0
RC                  0
Subs On             0
Min                 0
Mini Team Crest     0
Big Team Crest      0
Team Link           0
Team Link 2         0
Flag Code           0
TR                  0
dtype: int64


In [87]:
nanCountsPlayerPremierLeague = finalListPlayerPremierLeague.isna().sum()
print(nanCountsPlayerPremierLeague)

Id                  0
Player              0
Pos                 0
Age                 0
Height              0
Foot                0
Nation              0
Nation Flag Link    0
League              0
Team                0
Value               0
MP                  0
St                  0
G                   0
A                   0
G+A                 0
YC                  0
RC                  0
Subs On             0
Min                 0
Mini Team Crest     0
Big Team Crest      0
Team Link           0
Team Link 2         0
Flag Code           0
TR                  0
dtype: int64


In [88]:
nanCountsPlayerSerieA = finalListPlayerSerieA.isna().sum()
print(nanCountsPlayerSerieA)

Id                  0
Player              0
Pos                 0
Age                 0
Height              0
Foot                0
Nation              0
Nation Flag Link    0
League              0
Team                0
Value               0
MP                  0
St                  0
G                   0
A                   0
G+A                 0
YC                  0
RC                  0
Subs On             0
Min                 0
Mini Team Crest     0
Big Team Crest      0
Team Link           0
Team Link 2         0
Flag Code           0
TR                  0
dtype: int64


In [89]:
nanCountsPlayerLaLiga = finalListPlayerLaLiga.isna().sum()
print(nanCountsPlayerLaLiga)

Id                  0
Player              0
Pos                 0
Age                 0
Height              0
Foot                0
Nation              0
Nation Flag Link    0
League              0
Team                0
Value               0
MP                  0
St                  0
G                   0
A                   0
G+A                 0
YC                  0
RC                  0
Subs On             0
Min                 0
Mini Team Crest     0
Big Team Crest      0
Team Link           0
Team Link 2         0
Flag Code           0
TR                  0
dtype: int64


In [90]:
import json
import shutil
import os

dataClub = finalListClub.to_dict(orient='records')
dataClubList = []
dataClubList.append(dataClub)

dataClubSecond = finalListClubSecond.to_dict(orient='records')
dataClubListSecond = []
dataClubListSecond.append(dataClubSecond)

dataPlayer = finalListPlayer.to_dict(orient='records')
dataPlayerList = []
dataPlayerList.append(dataPlayer)

dataPlayerLigue1 = finalListPlayerLigue1.to_dict(orient='records')
dataPlayerLigue1List = []
dataPlayerLigue1List.append(dataPlayerLigue1)

dataPlayerBundesliga = finalListPlayerBundesliga.to_dict(orient='records')
dataPlayerBundesligaList = []
dataPlayerBundesligaList.append(dataPlayerBundesliga)

dataPlayerPremierLeague = finalListPlayerPremierLeague.to_dict(orient='records')
dataPlayerPremierLeagueList = []
dataPlayerPremierLeagueList.append(dataPlayerPremierLeague)

dataPlayerSerieA = finalListPlayerSerieA.to_dict(orient='records')
dataPlayerSerieAList = []
dataPlayerSerieAList.append(dataPlayerSerieA)

dataPlayerLaLiga = finalListPlayerLaLiga.to_dict(orient='records')
dataPlayerLaLigaList = []
dataPlayerLaLigaList.append(dataPlayerLaLiga)

data = dataClubList + dataClubListSecond + dataPlayerList + dataPlayerLigue1List + dataPlayerBundesligaList + dataPlayerPremierLeagueList + dataPlayerSerieAList + dataPlayerLaLigaList

with open('data.json', 'w', encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False)

source_path = 'data.json'
destination_path = os.path.join('src', 'data', 'data.json')

if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print(f"Successfully copied {source_path} to {destination_path}")
else:
    print("Source file does not exist!")

Successfully copied data.json to src/data/data.json
